In [1]:
from matplotlib import pylab
import nengo
import random
import numpy as np
import gzip as gz
import cPickle
from cPickle import load
try:
    import Image
except ImportError:
    from PIL import Image
from scipy.sparse.linalg import svds
import scipy
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.animation as animation

#%matplotlib inline #Makes visualizations appar inline (Commented out because animation popup as new window)

In [2]:
#The code in this cell is for reading the images from the MNIST database and not part of the brain model.

def load_img(path, dims):
    """Load the image at path and return an array representing the raster.
    Flattens image. Shifts pixel activations such that 0 represents gray,
    normalizes the output array.
    Keyword arguments:
    path -- str, path of the image to be loaded.
    dims -- (w, h), where w,h are ints indicating dimensions of the image (in
        px)."""

    img = Image.open(path).resize(dims).getdata()
    img.convert('L')
    img = subtract(array(img).flatten(), 127.5)
    return img/norm(img)


def load_data(filename):
    """Uncompress, unpickle and return a .pkl.gz file.
    Keyword arguments:
    filename -- str, a valid file path"""

    return load(gz.open(filename))

def load_mini_mnist(option=None):
    """Load and return the first \%10 of the images in the mnist dataset.
    Does not return labels. Pass in 'train', 'valid' or 'test' if you want to
    load a specific subset of the dataset.
    Keyword arguments:
    option -- str (default=None)."""

    mini_mnist = load(gz.open('./mini_mnist.pkl.gz', 'rb'))
    if option == 'train':
        return mini_mnist[0]
    elif option == 'valid':
        return mini_mnist[1]
    elif option == 'test':
        return mini_mnist[2]
    else:
        return mini_mnist

In [3]:
def rotate_img(img, degrees):
    '''
    img is the dim**2 by 1 vector representing the pixel values.
    Rotates image the degrees passed in counterclockwise
    Returns the Reshaped image (to original shape which is the one dimensional vector)
    dim is a global variable
    '''
    original = img.shape
    
    newImg = scipy.ndimage.interpolation.rotate(np.reshape(img, (dim,dim), 'F'),degrees,reshape=False)
    newImg = np.reshape(newImg, original, 'F')
    return newImg

In [4]:
conn_synapse = 0.1 #post synaptic time constant to use for filtering (pstc) - what does changing this do?
probe_synapse = 0.01 #pstc 
multiplier = 2 #not used
n_neurons = 5000
direct = True #Direct - function computed explicitly instead of in neurons 
stop_time = 3.0
run_time = 3.0 #in seconds

In [5]:
dim = 28 #size of the image
mnist = load_mini_mnist()
train = mnist[0] #collection of training images
img = mnist[1][0] #image to be used for testing
compress_size = 400 #?
basis, S, V = svds(train.T, k=compress_size) #Used for encoding and decoding information 
    #a set of vectors representing what a hand drawn number should look like?

In [6]:
#Need same number of vectors in basis as number of neurons (randomly sample from basis)
expanded_basis = np.array([random.choice(basis.T) for _ in range(n_neurons)]) 

In [7]:
def stim_func(t):
    '''returns the image for first 0.1s'''
    if t < 0.1:
        return img
    else:
        return [0 for _ in range(len(img))]

In [8]:
#Not currently being used
def stim_func_rot(t):
    '''Input used to control rate of rotation - scalar value.'''
    if t < 0.1:
        return 0
    else:
        return 1

In [9]:
def connection_func(x):
    '''takes the output from the first ensemble and rotates it by 1 degree or degrees specified by scalar input (depending on model)'''
    return rotate_img(x,1)
    #return rotate_img(x[:-1],x[-1]) #If using the scalar input
    #return rotate_img(x,10) - conn_synapse*(x) #Tried to erase the previous information instead of adding it

In [10]:
#A network is primarily used for grouping together related objects and connections for visualization purposes
with nengo.Network() as net:
    
    if direct:
        neuron_type = nengo.Direct() #function computed explicitly, instead of in neurons
    else:
        neuron_type = nengo.LIF() #spiking version of the leaky integrate-and-fire neuron model

    #Input stimulus (img) - provide data to the ensemble
    ipt = nengo.Node(stim_func)
    
    #Input scalar stimulus (rate of rotation)
    #ipt2 = nengo.Node(stim_func_rot)
    
    #Group of ensembles to represent each pixel individually (plus how many scalar inputs there are)
    ensArr = nengo.networks.EnsembleArray(100, dim**2+0, ens_dimensions=1) #incresing num neurons has small effect on run time
    
    
    
    #Group of neurons that collectively represent information(vector) - ensemble that brings pixels together
    ens = nengo.Ensemble(n_neurons,
                         dimensions=dim**2, #pixels of the image? 28*28
                         encoders=expanded_basis, #transform representational space to neuron space
                         eval_points=expanded_basis, #used for decoder solving, spanning interval
                         n_eval_points=n_neurons,
                         neuron_type=neuron_type)
    
    #Ensemble for viewing output
    ens2 = nengo.Ensemble(n_neurons,
                         dimensions=dim**2, #pixels of the image? 28*28
                         encoders=expanded_basis, #transform representational space to neuron space
                         eval_points=expanded_basis, #used for decoder solving, spanning interval
                         n_eval_points=n_neurons,
                         neuron_type=neuron_type)
    
    for i in range(ensArr.n_ensembles):
        #to run in direct or LIF
        ensArr.ea_ensembles[i].neuron_type = neuron_type
        #Connect the input (img) to each ensemble individually 
        nengo.Connection(ipt[i],ensArr.ea_ensembles[i])
        #Connect each ensemble to the same new ens
        nengo.Connection(ensArr.ea_ensembles[i],ens[i])
        #Connect each ensemble to another ens for viewing
        nengo.Connection(ensArr.ea_ensembles[i],ens2[i])
        #Connect the ens output to each ensemble individually
        nengo.Connection(ens[i],ensArr.ea_ensembles[i])
    
    #Connect input for rate of rotation to its own ensemble
    #nengo.Connection(ipt2,ensArr.ea_ensembles[-1])
    #nengo.Connection(ensArr.ea_ensembles[-1],ens[-1])
    
    
    #Recurrent connection in ens that performs the rotation and feeds back into ensemble
    nengo.Connection(ens,ens, synapse=conn_synapse, function=connection_func)
    
    #Collects data from the simulation
    probe = nengo.Probe(ens2, attr='decoded_output',#sample_every=0.001,
                       synapse=probe_synapse)
        

In [11]:
#Reference simulator for Nengo models.
#The simulator takes a Network and builds internal data structures to run the model defined by that network
sim = nengo.Simulator(net)

In [12]:
#Run the simulator
sim.run(run_time) #4:26 LIF and 1:29 Direct

Simulation finished in 0:01:29.                                                 


In [ ]:
#The following code is for viewing the probe output and is not part of the brain model.

In [147]:
pylab.imshow(np.reshape(img, (dim,dim), 'F'), cmap='Greys_r')

In [148]:
'''Image at stop time'''
pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][int(stop_time*1000)-1]], 
                             (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)

In [149]:
'''Image at start time'''
pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][1]], 
                             (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)

In [13]:
'''Animation for Probe output'''
fig = plt.figure()

def updatefig(i):
    im = pylab.imshow(np.reshape([0. if x < 0.00001 else x for x in sim.data[probe][i]],
                                 (dim, dim), 'F'), cmap=plt.get_cmap('Greys_r'),animated=True)
    
    return im,

ani = animation.FuncAnimation(fig, updatefig, interval=1, blit=True)
plt.show()


In [14]:
# save the output
#cPickle.dump(sim.data[probe], open( "Buffer_rotations_in_connection_with_ens_arr_direct.p", "wb" ) )